In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals

In [2]:
from pymorphy2 import MorphAnalyzer
import collections
import codecs
import re
import string
import stop_words
import pandas as pd

In [3]:
class Google:
    __EXCLUDE_SYMBOLS_STR = u''.join(['№', '«', 'ђ', '°', '±', '‚', 'ћ', '‰', '…', '»', 'ѓ', 'µ', '·', 'ґ', 'њ', 'ї', 'џ', 'є', '‹',
                                '‡', '†', '¶', 'ќ', '€', '“', 'ў', '§', '„', '”', '\ufeff', '’', 'љ', '›', '•', '—', '‘', 
                                '\x7f', '\xad', '¤', '\xa0'])
            
    __regex_punct = re.compile('[%s]' % re.escape(string.punctuation))
    __regex_dig = re.compile('[%s]' % re.escape(string.digits))
    __regex_symb = re.compile('[%s]' % re.escape(__EXCLUDE_SYMBOLS_STR))
    __regex_struct = re.compile('[%s]' % string.printable + string.whitespace)
    __emoji_pattern = re.compile("["
                                 "\U0001F600-\U0001F64F"  # emoticons
                                 "\U0001F300-\U0001F5FF"  # symbols & pictographs
                                 "\U0001F680-\U0001F6FF"  # transport & map symbols
                                 "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                 "]+")
    __morph = MorphAnalyzer()
    
    def __init__(self, post_proc_csv='laba2.csv',
                 raw_data_csv='laba1.csv', token_dict_file='common.dict',
                 n=10):
        self.__n = n
        self.__post_proc_data = pd.read_csv(post_proc_csv, header=[0, 1], index_col=0, encoding='utf-8')

        self.__raw_data = pd.read_csv(raw_data_csv, header=[0, 1], index_col=0, encoding='utf-8')

        with codecs.open(token_dict_file, 'r', 'utf-8') as f:
            self.__token_dict = {line.split()[0]: line.split()[2] for line in f}

        self.__invert_index = collections.defaultdict(list)
        for token, ind in self.__token_dict.items():
            for category in self.__post_proc_data.columns.levels[0]:
                for index, item in enumerate(self.__post_proc_data[category]['Text']):
                    if ind in item.split():
                        self.__invert_index[token].append((category, index))

    def __preprocessor(self, query):
        query = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', u'', query)
        query = re.sub(' +' , ' ', query)
        query = query.strip().strip('\t').replace('\n', u'')
        query = self.__regex_punct.sub('', query)
        query = self.__regex_dig.sub('', query)
        query = self.__regex_symb.sub(' ', query)
        query = self.__regex_struct.sub('', query)
        query = re.sub(' +' , ' ', query.strip())
        query = self.__emoji_pattern.sub('', query)
        query = query.lower().split()
        tokens = []
        for word in query:
            if word not in stop_words.get_stop_words('ru'):
                tokens.append(word)
        return [self.__morph.parse(word)[0].normal_form for word in tokens]
    
    def __unique_list(self, l):
        x = []
        for a in l:
            if a not in x:
                x.append(a)
        return x
    
    def __get_indexes(self, tokens):
        indexes = set(self.__invert_index[tokens[0]])
        for token in tokens[1:]:
            indexes.intersection_update(set(self.__invert_index[token]))
        return indexes
    
    def find(self, query):
        prep_tokens = self.__preprocessor(query)
        indexes = list(self.__get_indexes(prep_tokens))
        if len(indexes) < self.__n:
            combinations = [prep_tokens]
            flag = False
            for item in combinations:
                if flag or len(item) <= 1:
                    break
                for ind, elem in enumerate(item):
                    tmp = item[:]
                    tmp.pop(ind)
                    combinations.append(tmp)
                    indexes.extend(list(self.__get_indexes(tmp)))
                    indexes = self.__unique_list(indexes)
                    if len(indexes) >= self.__n:
                        f = True
                        break
        indexes = indexes[:self.__n]
        texts = []
        for index in indexes:
            texts.append({u'Заголовок': self.__raw_data[index[0]]['Title'][index[1]],
                          u'Текст': self.__raw_data[index[0]]['Text'][index[1]]})
        return texts

In [4]:
google = Google()

In [7]:
texts = google.find(u"Кассовые сборы фильмов в Российском прокате")

In [8]:
print u"Найдено", len(texts), u"статей"
for i, item in enumerate(texts):
    print "________________________________________________________________________________________________________________"
    print u"###Заголовок###"
    print item[u'Заголовок']
    print "________________________________________________________________________________________________________________"
    print u"###Текст###"
    print item[u'Текст']

Найдено 10 статей
________________________________________________________________________________________________________________
###Заголовок###
Фильм "Т-34" уже посмотрели полтора миллиона человек
________________________________________________________________________________________________________________
###Текст###
В настоящий момент кассовые сборы новой отечественной драмы составляют более 500 миллионов рублей.

Более полутора миллионов человек посмотрели фильм Алексея Сидорова "Т-34". Такой результат продемонстрирован был за первые три дня полноценного проката — с 1 по 3 января 2019 года.

Информацию о показателях фильма в прокате сообщила пресс-служба Министерства культуры Российской Федерации, при поддержке которого и снимался "Т-34". Что же касается кассовых сборов, то в настоящий момент этот показатель приближается к отметке в 520 миллионов рублей.

Напомним, "Т-34" — новый отечественный военный фильм, который рассказывает драматическую историю о том, как узник концлагеря